## TODO
* Re-write for more than two classes
* First fix FI, then FM
* Monte Carlo sim to test all permutations; OR:
* Increment steps based on PDF, with each step having an equal prob.
* Jim's suggestion: try replacing EM with land, in addition to water, as I have it now
* Add randomization/monte-carlo into bins- instead of just using mean values (i.e. for size bins, or for FLUX_STEP in senstivity)
* For typical EM fraction: use regional sub-binning instead of UAVSAR footprint 
* Use typical regional values for EM fluxes, the way I use a regional LSD and inun fraction
* install add-ons: git, variable inspector: first install node.js, npm, jupyterlab-git
* Find a plugin to make matplotlib plots interactive like when they appear in sep. window
* make dimensionless version
* (contour plot: fill in color between lines) DONE
* Add size binning: ingest lake size distrib DONE

## Description
This script takes as input the landscape inundation fraction, the emergent macrophyte fraction (fraction of all inundated areas), and flux values for these fractions. It then iterates over probable ranges and plots the total flux's sensitivities to these values, Normalized by area.


| Name  | Description | Value |
|---      |---                |---       |
| INPUTS |
FLUXES | Dictionary of flux values by class	<mg CH4 / m2/ day>| OW: (0, 166); EM: (77, 5800)
FRACTION:IN_TOTAL | Fraction inundated: Range for open water and emergent macrophyte area divided by total landscape area | (10, 38)
FRACTION:EM | Fraction EMERGENT macrophyte: Range for emergent macrophyte area / open water area | (2, 60)
FRACTION:  ... | ...
FRACTION_STEPS | Number of steps for each landcover type (range / increment) |
FLUX_STEPS | Number of steps for flux (range / increment) |


In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import Colormap
import matplotlib
import matplotlib.mlab as mlab
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import scipy.ndimage as ndi 
from scipy.io import loadmat
from scipy.stats import binned_statistic
import pandas as pd
# %matplotlib inline

### Inputs

In [26]:
# fluxes={'OW': 100,
#        'GW': 300,
#        'SW': 200}

FRACTION={
    'IN_TOTAL':(
    (22, 31),
    (10,16),
    (27, 30),
    (37.4, 37.6),
    (10, 37.6)), # Default Fraction Inundated; From Run 35
    'EM':((35, 58),
     (42, 60),
     (12, 21),
     (2, 2.5),
     (0, 100)) #(2, 30) # Default Fraction macrophyte: From Run 35 (range: 2:60)
}
FLUXES={
    'OW': (0, 166), # Open water; From range in: Burger et al 2016	Biogeosciences | Superceded if using LSD
    'EM': (77, 2000) # (77, 3000) #(77, 380)  # Emergent macrophyte; From mean (max = 5800) in: Burger et al 2016	Biogeosciences
}
    # Sensitivity raster dimensions
FRACTION_STEPS=40
FLUX_STEPS=40
SITES=('PAD','YFB','YKN','DAR','All') #short names for sites
SITES_LSD = (9,2,7,7,0) # numbers as givin in Lake Size Distribution files
# REGION=0 # Set to -1 if using default; 0=PAD

# LAT=59 # latitude, in degrees (PAD=59)

### Lake size distribution inputs

Labels:
    'All'
    'Sagavanirktok River'
    'Yukon Flats Basin'
    'Old Crow Flats'
    'Mackenzie River Delta'
    'Mackenzie River Valley'
    'Canadian Shield Margin'
    'Canadian Shield'
    'Slave River'
    'Peace-Athabasca Delta'
    'Athabasca River'
    'Prairie Potholes North'
    'Prairie Potholes South'
    'Tuktoyaktuk Peninsula'
    []
    []
    []
    []
    []
    []
    []
    'Prairie pothole regions'
    'Canadian Shield'
    'Thermokarst regions'
    'Arctic-boreal wetlands'
    'Lowland river valleys'

In [27]:
lsd_pth='/mnt/d/GoogleDrive/Research/Lake distributions/savedData/fitting_data_regional_4_det.mat' # these directories hold the final version of data used for 2019 paper
lsd_labels_pth='/mnt/d/GoogleDrive/Research/Lake distributions/regionLabels4.mat'

### Size binning data from Holgerson and Raymond 2016

In [28]:
BIN_EDGES=np.array([0, 0.001, 0.01, 0.1, 1, 10, 100, 1000000]) # <km2>
# BIN_EDGES=np.array([0.001, 0.01, 0.1, 1, 10, 100, 10000000]) * 1000000 # <km2> * 1000000 = <m2>
BIN_FLUXES=np.array([2.28, 0.65, 0.28, 0.16,  0.12, 0.10, 0.06])*16.043 # <mmolC (in CH4)/m2/d> * 16.043 = <mg CH4/m2/d>
len(BIN_FLUXES)

7

### Load data

In [29]:
fused_regional=loadmat(lsd_pth, variable_names='Fused_regional', squeeze_me=True)

##############
# for area for each region, see: D:\GoogleDrive\Research\Lake distributions\savedData\out
##############

### Create sensitivity raster

In [30]:
SENSITIVITY={}
FRACTION_POSTINGS={}
FLUX_POSTINGS={}
for si, s in enumerate(SITES):
    print('\nSite: {}'.format(s))
    lsd = fused_regional['Fused_regional'][SITES_LSD[si]] # Lake size distribution for chosen region
    SENSITIVITY[s]=np.zeros((FRACTION_STEPS, FLUX_STEPS), 'double') # init
    # lsd_tmp=lsd # init
    FRACTION_POSTINGS[s]=np.linspace(FRACTION['EM'][si][0], FRACTION['EM'][si][1], FRACTION_STEPS)
    for Class in FLUXES.keys():
        FLUX_POSTINGS[Class]=np.linspace(FLUXES[Class][0], FLUXES[Class][1], FLUX_STEPS)
    lsd_vals = binned_statistic(lsd, lsd, statistic='sum', bins=BIN_EDGES)
    ow_flux_mean=np.sum(lsd_vals.statistic*BIN_FLUXES)/lsd.sum()
    print(f'Open water flux used: {ow_flux_mean:.3} [mg CH_4 / m^2 / day]')
    print(f'EM fraction: {0:0.2f}%\t| Total W area in first bin: \t\
        {lsd_vals.statistic[0]*(100- 0)/100:.2f}\t| Total W area: {np.sum(lsd)*(100- 0)/100:.2f}\n')
    for i in range(FRACTION_STEPS):
        ##############################################
             # toggle if not using LSD
    #     ow_flux_mean=np.mean(FLUX_POSTINGS['OW'])
        ##############################################
            # toggle if using LSD | with lakes changing bins # Results in higher water flux in highest EM bin bc more of the lakes are now small (unrealistic)
    #     lsd_tmp_vals = binned_statistic(lsd_tmp, lsd_tmp, statistic='sum', bins=BIN_EDGES) # could also use np.digitize
    #     ow_flux_mean=np.sum(lsd_tmp_vals.statistic*BIN_FLUXES)/lsd_tmp.sum()
    #     print(f'EM fraction: {FRACTION_POSTINGS[i]:0.2f}%\t| Total W area in first bin: \
    #         {lsd_tmp_vals.statistic[0]:.2f}\t| Total W area: {np.sum(lsd_tmp_vals.statistic):.2f}\t| Total W flux: {((100- FRACTION_POSTINGS[i]) * ow_flux_mean) / 100:.2f}')
        ##############################################
            # toggle if using LSD | with lakes NOT changing bins # Results in lower water flux in highest EM bin
        print(f'EM fraction: {FRACTION_POSTINGS[s][i]:0.2f}%\t| Total W area in first bin: \
            {lsd_vals.statistic[0]*(100- FRACTION_POSTINGS[s][i])/100:.2f}\t| Total W area: \
            {np.sum(lsd_vals.statistic)*(100- FRACTION_POSTINGS[s][i])/100:.2f}\t| Total W flux: {((100- FRACTION_POSTINGS[s][i]) * ow_flux_mean) / 100:.2f}')
        ##############################################
        for j in range(FLUX_STEPS):
            # each cell = <region's inundation fraction> * (<EM fraction> * <EM flux> + <OW fraction> * <OW flux>)
            SENSITIVITY[s][i,j] = np.mean(FRACTION['IN_TOTAL'][si]) / 100 * (FRACTION_POSTINGS[s][i] * FLUX_POSTINGS['EM'][j] + # use median? # note: hard-coded for just two classes: OW and EM
                                (100- FRACTION_POSTINGS[s][i]) * ow_flux_mean) / 100
        


Site: PAD
Open water flux used: 1.36 [mg CH_4 / m^2 / day]
EM fraction: 0.00%	| Total W area in first bin: 	        0.40	| Total W area: 186.62

EM fraction: 35.00%	| Total W area in first bin:             0.26	| Total W area:             121.30	| Total W flux: 0.89
EM fraction: 35.59%	| Total W area in first bin:             0.26	| Total W area:             120.20	| Total W flux: 0.88
EM fraction: 36.18%	| Total W area in first bin:             0.26	| Total W area:             119.10	| Total W flux: 0.87
EM fraction: 36.77%	| Total W area in first bin:             0.25	| Total W area:             118.00	| Total W flux: 0.86
EM fraction: 37.36%	| Total W area in first bin:             0.25	| Total W area:             116.90	| Total W flux: 0.85
EM fraction: 37.95%	| Total W area in first bin:             0.25	| Total W area:             115.80	| Total W flux: 0.85
EM fraction: 38.54%	| Total W area in first bin:             0.25	| Total W area:             114.70	| Total W flux: 0.84


## SCRAP

In [31]:
## SCRAP

FRACTION['EM'][si]
FLUXES

{'OW': (0, 166), 'EM': (77, 2000)}

In [32]:
## SCRATCH
i=1
    lsd-0.004
lsd_tmp=lsd*(100-FRACTION_POSTINGS[i])/100 # keeps decreasing in mag. each iteration as EM fraction grows
lsd
FLUX_POSTINGS['OW'][j]
FRACTION_POSTINGS
FLUX_POSTINGS['OW']
lsd_tmp_vals = binned_statistic(lsd_tmp, lsd_tmp, statistic='count', bins=BIN_EDGES) # could also use np.digitize, for binned_stat, need to add min and max bins on either end and rightmost bin is basically ignored
# np.any(np.isin(lsd_tmp, np.nan))
# lsd_tmp
# BIN_FLUXES
lsd_tmp_vals.statistic.shape
BIN_EDGES
# lsd_tmp_vals.statistic
# BIN_FLUXES.shape
# lsd_tmp_vals.statistic
lsd_tmp_vals.statistic*BIN_FLUXES
np.sum(lsd_tmp_vals.statistic*BIN_FLUXES)/lsd.sum()
# lsd_tmp.sum()
# np.sum(lsd_tmp >= BIN_EDGES[-2])
# BIN_EDGES[-2]
i=7
print(f'Bins: {BIN_EDGES}')
print(f'Number of lakes in alleged bin {i}: {np.sum((lsd_tmp > BIN_EDGES[i-1]) & (lsd_tmp <= BIN_EDGES[i]))}')
print(f'Number of lakes in statistic bins: {lsd_tmp_vals.statistic}')
np.max(lsd)
np.sum(lsd_tmp > BIN_EDGES[i])
i=39
(100-FRACTION_POSTINGS[i])/100
np.sum(lsd_vals.statistic)
lsd_vals.statistic
np.sum(lsd)*(100- 0)/100

IndentationError: unexpected indent (<ipython-input-32-94ddb254ca94>, line 3)

In [33]:
## Raster plot
# plt.xticks(FLUX_POSTINGS[0:4:])
# plt.yticks(FRACTION_POSTINGS[0:4:])
f=16 # font size
# ax=np.zeros((3,2))
shape=(3,2)
fig, ax = plt.subplots(shape[0], shape[1], sharex=True, figsize=(6,8))
for si, s in enumerate(SITES):
    idx=np.unravel_index(si, shape)
    p=ax[idx].imshow(SENSITIVITY[s], origin='lower', aspect='auto', cmap='gnuplot', vmin=0, vmax=450, extent=(FLUXES['EM'][0], FLUXES['EM'][1], FRACTION['EM'][si][0], FRACTION['EM'][si][1]))
#     ax[idx].set_xlabel("Littoral zone flux ($mg\ CH_4 / m^2 /day$)", fontsize=f)
#     ax[idx].set_ylabel('Vegetated fraction (%)', fontsize=f)
    fig.text(0.5, 0.04, "Littoral zone flux ($mg\ CH_4 / m^2 /day$)", ha='center', va='bottom', fontsize=f, FONTNAME='DejaVu Sans')
    fig.text(0.04, 0.5, 'Vegetated fraction (%)', va='center', rotation='vertical', fontsize=f, FONTNAME='sans-serif')
#     ax[idx].set(xlabel="Littoral zone flux ($mg\ CH_4 / m^2 /day$)", ylabel='Vegetated fraction (%)', fontsize=f)
#     ax[idx].set_title('{} | Total flux ($mg\ CH_4 / m^2 /day$)'.format(s), fontsize=f)
    ax[idx].set_title(f'{s}: {np.mean(SENSITIVITY[s]):0.0f}', fontsize=f)
#     ax[idx].colorbar()
#     fig.tight_layout()
#     axes=plt.gca()
    # plt.rcParams({'dpi':300})
    # axes.set_xlim(FLUXES['EM'])
    # axes.set_ylim(FRACTION['EM'])
    # plt.xticks(FLUX_POSTINGS[0:4:])
    # plt.yticks(FRACTION_POSTINGS[0:4:])

    ## calculate median:
    cy, cx = ndi.center_of_mass(SENSITIVITY[s])

    ## report mean value:
#     print('Mean flux: \t{:.1f} mg CH_4/m^2/day'.format(np.mean(SENSITIVITY[s])))
#     print('\'Median\' flux: \t{:.1f} mg CH_4/m^2/day'.format(SENSITIVITY[s][cy.astype(int),cx.astype(int)])) # only equal to median if each step size is of equal prob.
#     print('''\nINPUTS:
#     FRACTION:\t{}
#     FLUXES:\t\t{}
#     '''.format(FRACTION, FLUXES))

    # common plot options
fig.suptitle('Water surface flux contribution ($mg\ CH_4 / m^2 /day$)', fontsize=f+3)
ax[shape[0]-1, shape[1]-1].axis('off')
# ax[shape[0]-1, shape[1]-1].colorbar()
cbaxes = fig.add_axes([0.6, 0.2, 0.28, 0.03]) 
fig.colorbar(p, cax=cbaxes, orientation='horizontal')
# fig.colorbar(p, ax=ax[shape[0]-1, shape[1]-1], orientation='horizontal')
figPth='/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRaster.pdf'
plt.savefig(figPth, dpi=200)
print('Saved fig to:\t{}'.format(figPth))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved fig to:	/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRaster.pdf


In [10]:
np.mean(SENSITIVITY[s])

113.76668318959445

In [11]:
ax.flat

In [34]:
## Make contour plot
## TODO: fill in between lines
## from https://matplotlib.org/examples/pylab_examples/contour_label_demo.html

    # I/O
num_contours=10

matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

##
plt.figure()

# Define a class that forces representation of float to look a certain way
# This remove trailing zero so '1.0' becomes '1'
class nf(float):
    def __repr__(self):
        str = '%.1f' % (self.__float__(),)
        if str[-1] == '0':
            return '%.0f' % self.__float__()
        else:
            return '%.1f' % self.__float__()
    # axis
fig, ax = plt.subplots(shape[0], shape[1], sharex=True, figsize=(6,8))

    # quick fix
# quick_title=SITES # keep unchanged
quick_title=['PAD','YF','CS-1','CS-2','All'] # keep unchanged

    # loop
for si, s in enumerate(SITES):
    idx=np.unravel_index(si, shape)
        
        # define dims
    Y=FRACTION_POSTINGS[s]
    X=FLUX_POSTINGS['EM']
    Z=SENSITIVITY[s]
    
    # Basic contour plot
    CS = ax[idx].contour(X, Y, Z, num_contours, cmap='gnuplot', vmin =0, vmax=500,
        extent=(FLUXES['EM'][0], FLUXES['EM'][1], FRACTION['EM'][si][0], FRACTION['EM'][si][1]))

    # Recast levels to new class
    CS.levels = [nf(val) for val in CS.levels]

    # Label levels with specially formatted floats
    if plt.rcParams["text.usetex"]:
        fmt = r'%r'
    else:
        fmt = '%r'
    ax[idx].clabel(CS, CS.levels, inline=True, fmt=fmt, fontsize=10)
#     ax[idx].set_xlabel("Littoral zone flux ($mg\ CH_4 / m^2 /day$)", fontsize=f+2)
#     ax[idx].set_ylabel('Vegetated fraction (%)', fontsize=f)
    fig.text(0.5, 0.04, "Littoral zone flux ($mg\ CH_4 / m^2 /day$)", ha='center', va='bottom', fontsize=f, FONTNAME='DejaVu Sans')
    fig.text(0.04, 0.5, 'Vegetated fraction (%)', va='center', rotation='vertical', fontsize=f, FONTNAME='sans-serif')
    ax[idx].set_title(f'{quick_title[si]}: {np.mean(SENSITIVITY[s]):0.0f}', fontsize=f)
#     plt.tight_layout()
    
    # plot format
plt.suptitle('Water surface flux contribution ($mg\ CH_4 / m^2 /day$)', fontsize=f)
ax[shape[0]-1, shape[1]-1].axis('off')

    # save
figPth='/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRasterContours.pdf'
plt.savefig(figPth, dpi=200)
figPth='/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRasterContours.jpg'
plt.savefig(figPth, dpi=200)
print('Saved fig to:\t{}'.format(figPth))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved fig to:	/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRasterContours.jpg


In [163]:
## Add fluxes from lit
lit_pth='data/Flux_synthesis.xlsx'
df=pd.read_excel(lit_pth)
df=df[~np.isnan( df['flux_to_plot_[mgCH4_m2_day]'])]

## fill empty vals
df['confidence_int_min_[mgCH4_m2_day]'][np.isin(df['confidence_int_min_[mgCH4_m2_day]'], 0)]=np.mean(df['confidence_int_min_[mgCH4_m2_day]'])
df['confidence_int_max_[mgCH4_m2_day]'][np.isin(df['confidence_int_max_[mgCH4_m2_day]'], 0)]=np.mean(df['confidence_int_max_[mgCH4_m2_day]'])
df['EM_percent_conf_in_min'].loc[np.isnan(df['EM_percent'])]=50 # arbitrary value for lit fluxes CI w no EM % data
df['EM_percent_conf_in_max'].loc[np.isnan(df['EM_percent'])]=50 # arbitrary value for lit fluxes CI w no EM % data
df['EM_percent'].loc[np.isnan(df['EM_percent'])]=50 # arbitrary value for lit fluxes w no EM % data

df['EM_percent_conf_in_min'].loc[np.isnan(df['EM_percent_conf_in_min'])]=5 # arbitrary value, placeholder for now #  | np.isin(df['EM_percent_conf_in_min'], 0)
df['EM_percent_conf_in_max'].loc[np.isnan(df['EM_percent_conf_in_max'])]=5 # arbitrary value, placeholder for now

## assign: lit vals
FLUXES_LIT=df[df['Orig_paper']!='above2_team' ]['flux_to_plot_[mgCH4_m2_day]'] # use for sens. plot
FRACTION_LIT=df[df['Orig_paper']!='above2_team' ]['EM_percent'] # use for sens. plot
FLUXES_LIT_CI=np.array([df[df['Orig_paper']!='above2_team' ]['confidence_int_min_[mgCH4_m2_day]'], df[df['Orig_paper']!='above2_team' ]['confidence_int_max_[mgCH4_m2_day]']]) # use for sens. plot
FRACTION_LIT_CI=np.array([df[df['Orig_paper']!='above2_team' ]['EM_percent_conf_in_min'],df[df['Orig_paper']!='above2_team' ]['EM_percent_conf_in_max']]) # use for sens. plot

## assign: field vals
FLUXES_FIELD=df[df['Orig_paper']=='above2_team' ]['flux_to_plot_[mgCH4_m2_day]'] # use for sens. plot
FRACTION_FIELD=df[df['Orig_paper']=='above2_team' ]['EM_percent'] # use for sens. plot
FLUXES_FIELD_CI=np.array([df[df['Orig_paper']=='above2_team' ]['confidence_int_min_[mgCH4_m2_day]'], df[df['Orig_paper']=='above2_team' ]['confidence_int_max_[mgCH4_m2_day]']]) # use for sens. plot
FRACTION_FIELD_CI=np.array([df[df['Orig_paper']=='above2_team' ]['EM_percent_conf_in_min'],df[df['Orig_paper']=='above2_team' ]['EM_percent_conf_in_max']]) # use for sens. plot

## view
df.head()
# df
# df.describe()

,Topic\Paper,Citation,Journal,Orig_paper,Typical flux (gC/m2/yr),flux_to_plot_[mgCH4_m2_day],confidence_int_min_[mgCH4_m2_day],confidence_int_max_[mgCH4_m2_day],Lake,Surface,EM_percent,EM_percent_conf_in_min,EM_percent_conf_in_max,Notes,flux_to_plot_[mmolCH4_m2_day],confidence_int_min_[mmolCH4_m2_day],confidence_int_max_[mmolCH4_m2_day]
27,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,21.49762,6.09634,6.09634,Dillon,OW,50.0,50.0,50.0,Open water Dillon. See Google Earth for locati...,1.34,0.38,0.38
28,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,1.12301,0.32086,0.32086,Long,OW,50.0,50.0,50.0,Open water Long,0.07,0.02,0.02
29,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,28.23568,8.82365,8.82365,Pass,OW,50.0,50.0,50.0,Open water Pass,1.76,0.55,0.55
30,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,47.16642,14.91999,14.91999,Red rock,OW,60.0,5.0,5.0,Open water Red rock,2.94,0.93,0.93
31,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,28.87740,8.18193,8.18193,Rainbow,OW,50.0,50.0,50.0,Open water Rainbow,1.80,0.51,0.51


In [164]:
df[np.isin(df['EM_percent'], 0)]
df
# df['EM_percent_conf_in_min'].loc[np.isnan(df['EM_percent'])]='oij'
# np.isnan(df['EM_percent_conf_in_min']) | np.isin(df['EM_percent_conf_in_min'], 0)
df

,Topic\Paper,Citation,Journal,Orig_paper,Typical flux (gC/m2/yr),flux_to_plot_[mgCH4_m2_day],confidence_int_min_[mgCH4_m2_day],confidence_int_max_[mgCH4_m2_day],Lake,Surface,EM_percent,EM_percent_conf_in_min,EM_percent_conf_in_max,Notes,flux_to_plot_[mmolCH4_m2_day],confidence_int_min_[mmolCH4_m2_day],confidence_int_max_[mmolCH4_m2_day]
27,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,21.49762,6.096340,6.096340,Dillon,OW,50.000000,50.0,50.0,Open water Dillon. See Google Earth for locati...,1.34,0.380,0.380
28,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,1.12301,0.320860,0.320860,Long,OW,50.000000,50.0,50.0,Open water Long,0.07,0.020,0.020
29,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,28.23568,8.823650,8.823650,Pass,OW,50.000000,50.0,50.0,Open water Pass,1.76,0.550,0.550
30,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,47.16642,14.919990,14.919990,Red rock,OW,60.000000,5.0,5.0,Open water Red rock,2.94,0.930,0.930
31,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,28.87740,8.181930,8.181930,Rainbow,OW,50.000000,50.0,50.0,Open water Rainbow,1.80,0.510,0.510
32,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,0.16043,0.160430,0.160430,Dillon,WM,50.000000,50.0,50.0,Wet meadow: Dillon,0.01,0.010,0.010
33,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,13.79698,3.048170,3.048170,Long,WM,50.000000,50.0,50.0,Wet meadow: Long,0.86,0.190,0.190
34,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,1.60430,0.641720,0.641720,Pass,WM,50.000000,50.0,50.0,Wet meadow: Pass,0.10,0.040,0.040
35,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,87.59478,8.823650,8.823650,Red rock,WM,60.000000,5.0,5.0,Wet meadow: Red rock,5.46,0.550,0.550
36,Seasonality of methane emissions from five lak...,"Smith and Lewis, 1992",GBC,NaN,NaN,202.14180,75.402100,75.402100,Red rock,NL,60.000000,5.0,5.0,Nuphar lutea: Red rock,12.60,4.700,4.700


In [149]:
## Scrap

# df[np.isnan(df['confidence_int_[mgCH4_m2_day]'])]
# df['confidence_int_[mgCH4_m2_day]']
# np.mean(df['confidence_int_[mgCH4_m2_day]'])
df[np.isin(df['confidence_int_min_[mgCH4_m2_day]'], 0)]
df[np.isin(df['confidence_int_min_[mgCH4_m2_day]'], 0)]['confidence_int_min_[mgCH4_m2_day]']
# df.loc[:, ([np.isin(df['confidence_int_min_[mgCH4_m2_day]'], 0)], df['confidence_int_min_[mgCH4_m2_day]'])]
df.loc[np.isin(df['confidence_int_min_[mgCH4_m2_day]'], 0), :]
np.isin(df['confidence_int_min_[mgCH4_m2_day]'], 0)
FLUXES_LIT_CI
FRACTION_LIT


27    50.000000
28    50.000000
29    50.000000
30    60.000000
31    50.000000
32    50.000000
33    50.000000
34    50.000000
35    60.000000
36    60.000000
37    50.000000
38    50.000000
39    50.000000
40    50.000000
41    50.000000
42    50.000000
58    50.000000
59    50.000000
60    50.000000
61    50.000000
62    50.000000
63    50.000000
64    50.000000
65    50.000000
66    50.000000
67    50.000000
68    50.000000
69    50.000000
86    87.616959
87    87.616959
88    50.000000
89    50.000000
90    50.000000
91    50.000000
Name: EM_percent, dtype: float64

In [165]:
## Single contour plot
%matplotlib widget
f=12 # fontsize
# num_contours = np.linspace(5, np.max(SENSITIVITY['All']), 11) #10
num_contours = np.linspace(2, 250, 15) #10

# Define a class that forces representation of float to look a certain way
# This remove trailing zero so '1.0' becomes '1'
class nf(float):
    def __repr__(self):
        str = '%.1f' % (self.__float__(),)
        if str[-1] == '0':
            return '%.0f' % self.__float__()
        else:
            return '%.1f' % self.__float__()
    # axis
fig, ax = plt.subplots(sharex=True, figsize=(4,4))

    # quick fix
# quick_title=SITES # keep unchanged
quick_title=['PAD','YF','CS-1','CS-2','All'] # keep unchanged

    # loop
for si, s in zip([4], ['All']):
        
        # define dims
    Y=FRACTION_POSTINGS[s]
    X=FLUX_POSTINGS['EM']
    Z=SENSITIVITY[s]
    
    # Basic contour plot
    CS = ax.contour(X, Y, Z, num_contours, cmap='gnuplot', vmin =0, vmax=500,
        extent=(FLUXES['EM'][0], FLUXES['EM'][1], FRACTION['EM'][si][0], FRACTION['EM'][si][1]))

    # Recast levels to new class
    CS.levels = [nf(val) for val in CS.levels]

    # Label levels with specially formatted floats
    if plt.rcParams["text.usetex"]:
        fmt = r'%r'
    else:
        fmt = '%r'
    ax.clabel(CS, CS.levels, inline=True, fmt=fmt, fontsize=10)
    ax.set_xlabel("Littoral zone flux ($mg\ CH_4 / m^2 /day$)", fontsize=f+2)
    ax.set_ylabel('Vegetated fraction (%)', fontsize=f)
#     fig.text(0.5, 0.04, "Littoral zone flux ($mg\ CH_4 / m^2 /day$)", ha='center', va='bottom', fontsize=f, FONTNAME='DejaVu Sans')
#     fig.text(0.04, 0.5, 'Vegetated fraction (%)', va='center', rotation='vertical', fontsize=f, FONTNAME='sans-serif')
#     ax.set_title(f'{quick_title[si]}: {np.mean(SENSITIVITY[s]):0.0f}', fontsize=f)
    fig.tight_layout()
    
## Superimpose errorbar plot: lit values
ax.errorbar(FLUXES_LIT, FRACTION_LIT, FRACTION_LIT_CI,FLUXES_LIT_CI, 'ko', ecolor='gray', fillstyle='none', alpha=0.6) # , clip_box=(0,0,6000,100)

## Superimpose errorbar plot: field values
ax.errorbar(FLUXES_FIELD, FRACTION_FIELD, FRACTION_FIELD_CI,FLUXES_FIELD_CI, 'bo', ecolor='#2D7DC4', fillstyle='none', alpha=0.6) # , clip_box=(0,0,6000,100)

## title
plt.title('Water surface flux contribution ($mg\ CH_4 / m^2 /day$)', fontsize=f-2)

## set window
ax.set_xlim(0, 1000)

    # save
figPth='/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRasterContoursErrors.jpg'
plt.savefig(figPth, dpi=200)
figPth='/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRasterContoursErrors.pdf'
plt.savefig(figPth, dpi=200)

print('Saved fig to:\t{}'.format(figPth))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Saved fig to:	/mnt/d/pic/UAVSAR_classification/upscaleSensitivityRasterContoursErrors.pdf


In [49]:
SENSITIVITY
# ?plt.plot
# plt.plot

{'PAD': array([[  7.3764061 ,  11.94969456,  16.52298302, ..., 176.58807917,
         181.16136763, 185.7346561 ],
        [  7.49461425,  12.14496178,  16.79530931, ..., 179.55747302,
         184.20782056, 188.85816809],
        [  7.6128224 ,  12.340229  ,  17.06763561, ..., 182.52686687,
         187.25427348, 191.98168009],
        ...,
        [ 11.75010764,  19.1745818 ,  26.59905596, ..., 286.45565162,
         293.88012579, 301.30459995],
        [ 11.86831579,  19.36984902,  26.87138226, ..., 289.42504547,
         296.92657871, 304.42811194],
        [ 11.98652394,  19.56511625,  27.14370855, ..., 292.39443932,
         299.97303163, 307.55162394]]),
 'YFB': array([[  4.47010956,   7.16230956,   9.85450956, ..., 104.08150956,
         106.77370956, 109.46590956],
        [  4.51419357,   7.23597819,   9.9577628 , ..., 105.22022434,
         107.94200896, 110.66379357],
        [  4.55827758,   7.30964681,  10.06101604, ..., 106.35893912,
         109.11030835, 111.86167758],

In [524]:
FRACTION_POSTINGS

{'All': array([ 2.        ,  3.48717949,  4.97435897,  6.46153846,  7.94871795,
         9.43589744, 10.92307692, 12.41025641, 13.8974359 , 15.38461538,
        16.87179487, 18.35897436, 19.84615385, 21.33333333, 22.82051282,
        24.30769231, 25.79487179, 27.28205128, 28.76923077, 30.25641026,
        31.74358974, 33.23076923, 34.71794872, 36.20512821, 37.69230769,
        39.17948718, 40.66666667, 42.15384615, 43.64102564, 45.12820513,
        46.61538462, 48.1025641 , 49.58974359, 51.07692308, 52.56410256,
        54.05128205, 55.53846154, 57.02564103, 58.51282051, 60.        ])}

In [15]:
CS.levels

[0, 8, 16, 24, 32, 40, 48, 56]